In [1]:
from geopandas.tools import sjoin
import os
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
import sys

wor_dir="C:\\Users\\ATeklesadik\\OneDrive - Rode Kruis\\Documents\\documents\\Typhoon-Impact-based-forecasting-model"

os.chdir(wor_dir)


### Source for pre disaster indicators: https://data.humdata.org/dataset/philippines-pre-disaster-indicators

In [7]:
file_name = "IBF-Typhoon-model\\data\\pre_disaster_indicators\\190108_pantawid-pamilya-beneficiary-households-and-household-members-by-sex-and-age-group.xlsx"

path = os.path.join(wor_dir, file_name)

pantawid_pamilya_beneficiary = pd.read_excel(path, sheet_name="Data", engine="openpyxl").filter(['PCODE_MUN_CTY','Total # of Active HHs','Total HH MEM_M','Total HH MEM_F'])
#pantawid_pamilya_beneficiary['Municipality_City Code']=pantawid_pamilya_beneficiary['PCODE_MUN_CTY']

pantawid_pamilya_beneficiary.rename(columns={"PCODE_MUN_CTY": "Mun_Code"},inplace=True)

display(pantawid_pamilya_beneficiary.head())

,Mun_Code,Total # of Active HHs,Total HH MEM_M,Total HH MEM_F
0,PH133902000,101,269,271
1,PH133908000,201,443,452
2,PH133909000,252,642,661
3,PH133910000,989,2939,2871
4,PH133911000,1038,2930,2929


### population per admin 

In [8]:
file_name = "IBF-Typhoon-model\\data\\pre_disaster_indicators\\180814_philippines-population-admin-1-to-3_2015_vertical.xlsx"

path = os.path.join(wor_dir, file_name)

population_admin = pd.read_excel(path, sheet_name="Municipality", engine="openpyxl").filter(['Municipality_City Code','Female','Male']).groupby('Municipality_City Code').agg('sum')

population_admin['Total Pop']=population_admin.sum(axis="columns") 
population_admin['Mun_Code']=population_admin.index.tolist()

population_admin.reset_index(drop=True, inplace=True)

#population_admin.rename(columns={"Municipality_City Code": "Mun_Code"},inplace=True)
display(population_admin.head())

,Female,Male,Total Pop,Mun_Code
0,825,967,1792,PH012801000
1,16175,16040,32215,PH012802000
2,15564,16052,31616,PH012803000
3,7422,7250,14672,PH012804000
4,27695,27506,55201,PH012805000


### vulnerable Groups

In [9]:
file_name = "IBF-Typhoon-model\\data\\pre_disaster_indicators\\180814_vulnerable-groups_by-city_municipality.xlsx"

path = os.path.join(wor_dir, file_name)

vulnerable_groups = pd.read_excel(path, sheet_name="Data", engine="openpyxl")

vulnerable_groups['vulnerable_groups']=vulnerable_groups.sum(axis="columns")
vulnerable_groups=vulnerable_groups.filter(['Municipality_City Code','vulnerable_groups'])
vulnerable_groups.rename(columns={"Municipality_City Code": "Mun_Code"},inplace=True)
display(vulnerable_groups.head())

C:\Users\ATEKLE~1\AppData\Local\Temp/ipykernel_24564/3066834646.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  vulnerable_groups['vulnerable_groups']=vulnerable_groups.sum(axis="columns")


,Mun_Code,vulnerable_groups
0,PH012801000,44
1,PH012802000,173
2,PH012803000,461
3,PH012804000,164
4,PH012805000,334


### housing material

In [10]:
file_name = "IBF-Typhoon-model\\data\\pre_disaster_indicators\\180814_construction-materials-of-the-outer-walls-and-roof_by-city_municipality.xlsx"

path = os.path.join(wor_dir, file_name)

construction_materials = pd.read_excel(path, sheet_name="by category", engine="openpyxl")
construction_materials.rename(columns={"Municipality_City Code": "Mun_Code"},inplace=True)
construction_materials=construction_materials.filter(['Mun_Code', 'Housing Units','Strong Roof/Strong Wall', 'Strong Roof/Light Wall','Strong Roof/Salvage Wall', 'Light Roof/Strong Wall','Light Roof/Light Wall', 'Light Roof/Salvage Wall','Salvaged Roof/Strong Wall', 'Salvaged Roof/Light Wall', 'Salvaged Roof/Salvage Wall'])
display(construction_materials.head())

,Mun_Code,Housing Units,Strong Roof/Strong Wall,Strong Roof/Light Wall,Strong Roof/Salvage Wall,Light Roof/Strong Wall,Light Roof/Light Wall,Light Roof/Salvage Wall,Salvaged Roof/Strong Wall,Salvaged Roof/Light Wall,Salvaged Roof/Salvage Wall
0,PH150708000,2560,441,1230,0,127,757,0,0,0,0
1,PH150709000,3647,503,1853,1,24,1253,2,0,3,0
2,PH150702000,14751,4972,6284,24,103,3244,10,6,33,23
3,PH150710000,2747,510,1218,7,10,991,5,0,1,1
4,PH150712000,3921,347,1309,5,30,2215,3,0,5,6


### Data on topography related variables 

In [11]:
geo_variable = pd.read_csv("IBF-Typhoon-model\\data\\topography\\geo_variable.csv")
geo_variable=geo_variable.filter(['Mun_Code', 'mean_slope', 'mean_elevation_m','ruggedness_stdev', 'mean_ruggedness', 'slope_stdev', 'area_km2','poverty_perc', 'with_coast', 'coast_length', 'perimeter'])

display(geo_variable.head())

,Mun_Code,mean_slope,mean_elevation_m,ruggedness_stdev,mean_ruggedness,slope_stdev,area_km2,poverty_perc,with_coast,coast_length,perimeter
0,PH012801000,16.83,558.75,39.76,84.41,8.54,111,10.10,0,NaN,45998
1,PH012802000,2.13,23.80,12.14,12.05,2.49,55,2.07,1,4294.0,33314
2,PH012803000,7.37,97.70,39.58,37.37,8.27,81,4.99,1,15821.0,64986
3,PH012804000,11.94,220.47,41.11,59.74,8.83,115,3.75,1,11813.0,52052
4,PH012805000,5.23,91.41,23.81,27.49,5.18,158,3.22,0,NaN,66662


### Data for Land slide /storm surge vulnerable areas 

In [12]:


data_matrix_new_variables = pd.read_csv("IBF-Typhoon-model\\data\\landuse_stormsurge\\data_matrix_new_variables.csv")
data_matrix_new_variables=data_matrix_new_variables.filter(['Mun_Code', 'landslide_per', 'stormsurge_per', 'Bu_p_inSSA','Bu_p_LS', 'Red_per_LSbldg', 'Or_per_LSblg', 'Yel_per_LSSAb','RED_per_SSAbldg', 'OR_per_SSAbldg', 'Yellow_per_LSbl'])
display(data_matrix_new_variables.head())

,Mun_Code,landslide_per,stormsurge_per,Bu_p_inSSA,Bu_p_LS,Red_per_LSbldg,Or_per_LSblg,Yel_per_LSSAb,RED_per_SSAbldg,OR_per_SSAbldg,Yellow_per_LSbl
0,PH012801000,4.08,0.00,0.00,4.08,4.08,0.0,0.00,0.00,0.00,0.00
1,PH012802000,1.41,1.54,1.54,1.41,0.00,0.0,1.41,1.48,0.03,0.03
2,PH012803000,2.45,1.68,1.68,2.45,0.04,0.0,2.40,1.64,0.00,0.04
3,PH012804000,2.88,1.51,1.51,2.88,0.41,0.0,2.47,1.44,0.00,0.07
4,PH012805000,0.03,0.03,0.03,0.03,0.03,0.0,0.00,0.00,0.00,0.03


### Impact Data(Damage and Loss data)

In [2]:

#df_total["perc_loss"] = df_total.apply(lambda x: division(x["area_affected"], x["rice_area"]), axis=1).values
impact = pd.read_csv("IBF-Typhoon-model\\data\\Impact\\IMpact_data_philipines_SEP_2021.csv")
impact['typhoon']=impact["typhoon"].astype(str)  + impact["Year"].astype(str)
impact['typhoon']= impact['typhoon'].map(lambda x: x.lower())
impact.rename(columns={"pcode": "Mun_Code"},inplace=True)
typhoons_with_impact_data=list(np.unique(impact.typhoon))
display(impact.head())
 



,Id,Mun_Code,typhoon,Year,Totally,Partially,total
0,1,PH041005000,durian2006,2006,84.0,588.0,672.0
1,2,PH041006000,durian2006,2006,1.0,7.0,8.0
2,3,PH041010000,durian2006,2006,4.0,48.0,52.0
3,4,PH041014000,durian2006,2006,0.0,78.0,78.0
4,5,PH041020000,durian2006,2006,16.0,169.0,185.0


In [5]:
len(np.unique(impact.typhoon))

40

### number of Households per Manucipality 

In [16]:
file_name = "IBF-Typhoon-model\\data\\pre_disaster_indicators\\180814_number-of-household_by-city_municipality.xlsx"

path = os.path.join(wor_dir, file_name)

household_by_city_municipality = pd.read_excel(path, sheet_name="Data", engine="openpyxl").filter(['Municipality_City','Municipality_City Code','Number of Household'])
household_by_city_municipality.rename(columns={"Municipality_City Code": "Mun_Code"},inplace=True)
household_by_city_municipality=household_by_city_municipality.filter(['Mun_Code','Number of Household'])
display(household_by_city_municipality.head())


,Mun_Code,Number of Household
0,PH012801000,411
1,PH012802000,8269
2,PH012803000,7375
3,PH012804000,3568
4,PH012805000,12797


### historical Typhoons Methadata (name, start date, end date, land fall)

In [ ]:
typhoon_metadata_filename = os.path.join(wor_dir, "IBF-Typhoon-model/data/rainfall_data/input/metadata_typhoons.csv")
typhoon_metadata = pd.read_csv(typhoon_metadata_filename, delimiter=",")

typhoons = typhoon_metadata.typhoon.tolist() ## historical typhoon events 
municipalities=population_admin.Mun_Code.tolist() 

N_typh = len(typhoons)
N_mun = len(municipalities)

municipality_codes_full = np.repeat(municipalities, N_typh)
typhoons_full = typhoons * N_mun

### create data frame for all pre disaster indicators 

In [18]:
data_temp1 = {"Mun_Code": municipality_codes_full, "typhoon": typhoons_full}
df_temp_total = pd.DataFrame(data_temp1)

##data_matrix_new_variables
df_temp_total=df_temp_total.join(data_matrix_new_variables.set_index('Mun_Code'), on='Mun_Code')

#geo_variable
df_temp_total=df_temp_total.join(geo_variable.set_index('Mun_Code'), on='Mun_Code')
#construction_materials
df_temp_total=df_temp_total.join(construction_materials.set_index('Mun_Code'), on='Mun_Code')
#vulnerable_groups
df_temp_total=df_temp_total.join(vulnerable_groups.set_index('Mun_Code'), on='Mun_Code')
#population_admin
df_temp_total=df_temp_total.join(population_admin.set_index('Mun_Code'), on='Mun_Code')
#pantawid_pamilya_beneficiary
df_temp_total=df_temp_total.join(pantawid_pamilya_beneficiary.set_index('Mun_Code'), on='Mun_Code')
df_predisasters=df_temp_total.set_index(['Mun_Code', 'typhoon'])
display(df_predisasters.head())

,Mun_Code,typhoon,landslide_per,stormsurge_per,Bu_p_inSSA,Bu_p_LS,Red_per_LSbldg,Or_per_LSblg,Yel_per_LSSAb,RED_per_SSAbldg,...,Salvaged Roof/Strong Wall,Salvaged Roof/Light Wall,Salvaged Roof/Salvage Wall,vulnerable_groups,Female,Male,Total Pop,Total # of Active HHs,Total HH MEM_M,Total HH MEM_F
0,PH012801000,durian2006,4.08,0.0,0.0,4.08,4.08,0.0,0.0,0.0,...,0.0,0.0,0.0,44,825,967,1792,169.0,439.0,401.0
1,PH012801000,fengshen2008,4.08,0.0,0.0,4.08,4.08,0.0,0.0,0.0,...,0.0,0.0,0.0,44,825,967,1792,169.0,439.0,401.0
2,PH012801000,aere2011,4.08,0.0,0.0,4.08,4.08,0.0,0.0,0.0,...,0.0,0.0,0.0,44,825,967,1792,169.0,439.0,401.0
3,PH012801000,atsani2020,4.08,0.0,0.0,4.08,4.08,0.0,0.0,0.0,...,0.0,0.0,0.0,44,825,967,1792,169.0,439.0,401.0
4,PH012801000,bopha2012,4.08,0.0,0.0,4.08,4.08,0.0,0.0,0.0,...,0.0,0.0,0.0,44,825,967,1792,169.0,439.0,401.0


### Add Typhoon wind Data

In [36]:
#Adding the wind data

data_temp1 = {"Mun_Code": municipality_codes_full, "typhoon": typhoons_full}
df_total = pd.DataFrame(data_temp1)

df_total["vmax"] = ""
df_total["dis_track_min"] = ""

df_wind_final= pd.DataFrame(columns=["typhoon", "Mun_Code", "v_max","dis_track_min"])

for typhoon in typhoons:

    # Path to the rainfall excel sheet per typhoon
    wind_path = os.path.join(
        wor_dir,
        "IBF-typhoon-model\\data\\wind_data\\output",
        typhoon +
        "_windgrid_output.csv",
    )
    df_temp = pd.read_csv(wind_path)
    
    df_wind_single = pd.DataFrame({"Mun_Code": df_temp["adm3_pcode"],"v_max": df_temp["v_max"],
                                   "dis_track_min": df_temp["dis_track_min"] })
    df_wind_single['typhoon']=typhoon
    df_wind_final = df_wind_final.append(df_wind_single) 


### Add rainfall data

In [37]:
# Path to the rainfall excel sheet per typhoon
rain_path = os.path.join(wor_dir,"IBF-typhoon-model\\data\\rainfall_data\\rainfall_data.csv")
df_rain_final = pd.read_csv(rain_path)
 

### Merge rainfall and Wind data

In [38]:
df1=df_rain_final.set_index(['Mun_Code', 'typhoon'])
df2=df_wind_final.set_index(['Mun_Code', 'typhoon'])
df_hazard = df1.join(df2, on=['Mun_Code','typhoon'])
display(df_hazard.head())

,,rainfall_Total,rainfall_max_6h,rainfall_max_24h,v_max,dis_track_min
Mun_Code,typhoon,,,,,
PH175301000,durian2006,1.936667,0.832222,0.243611,NaN,NaN
PH175101000,durian2006,182.760714,14.716071,7.381696,55.032241,2.478142
PH030801000,durian2006,27.012500,1.893750,1.070833,23.402905,136.527982
PH021501000,durian2006,126.900000,7.708333,5.650532,NaN,NaN
PH083701000,durian2006,0.137500,3.563542,1.050260,8.728380,288.358553
...,...,...,...,...,...,...
PH097332000,noul2015,4.787931,0.218678,0.130029,NaN,NaN
PH074625000,noul2015,1.080000,0.048333,0.030000,NaN,NaN
PH034932000,noul2015,3.250000,0.391667,0.135417,NaN,NaN


### merge impact data with hazard(rainfall+wind speed) data

In [57]:
df2=impact.set_index(['Mun_Code', 'typhoon'])
df_impact_hazard = df_hazard.join(df2, on=['Mun_Code','typhoon'])
display(df_impact_hazard.head())

,,rainfall_Total,rainfall_max_6h,rainfall_max_24h,v_max,dis_track_min,id,Year,Totally,Partially,total
Mun_Code,typhoon,,,,,,,,,,
PH175301000,durian2006,1.936667,0.832222,0.243611,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PH175101000,durian2006,182.760714,14.716071,7.381696,55.032241,2.478142,30.0,2006.0,261.0,618.0,879.0
PH030801000,durian2006,27.012500,1.893750,1.070833,23.402905,136.527982,NaN,NaN,NaN,NaN,NaN
PH021501000,durian2006,126.900000,7.708333,5.650532,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PH083701000,durian2006,0.137500,3.563542,1.050260,8.728380,288.358553,NaN,NaN,NaN,NaN,NaN


###  Normalize data uisng population size or number of housing uints per manucipality (Calculate percentage per manucipality)

In [53]:
def set_zeros(x):
    x_max = 30
    y_max = 6
    
    v_max = x[0]
    rainfall_max_6h = x[1]
    Totally = x[2]
    dis_track_min=x[3]

    if pd.notnull(Totally):
        value = Totally

    #elif (v_max < np.sqrt(x_max ** 2 - rainfall_max_6h ** 2) * (y_max / x_max) ) & dis_track_min >100:
    elif ((v_max < x_max)  and  (rainfall_max_6h < y_max) ) or dis_track_min >100:
        value = 0

    else:
        value = np.nan

    return value

def division(x, y):
    try:
        value = (x / y)
    except:
        value = np.nan
    return value

df_total= df_impact_hazard.join(df_predisasters, on=['Mun_Code','typhoon'])

df_total=df_total.reset_index()
df_total=df_total[df_total.typhoon.isin(typhoons_with_impact_data)]
df_total=df_total[df_total['v_max'].notnull()]
df_total=df_total[df_total['rainfall_max_6h'].notnull()]
  

# Setting the new damage threshold
df_total["Totally"] = df_total[["v_max", "rainfall_max_6h", "Totally","dis_track_min"]].apply(set_zeros, axis="columns")

df_total["perc_dmg"] = df_total.apply(lambda x: division(x["Totally"], x["Housing Units"]), axis=1).values

df_total["vulnerable_groups"] = df_total.apply(lambda x: division(x["vulnerable_groups"], x["Total Pop"]), axis=1).values
df_total["pantawid_pamilya_beneficiary"] = df_total.apply(lambda x: division(x["Total # of Active HHs"], x["Housing Units"]), axis=1).values

df_total["Strong Roof/Strong Wall"] = df_total.apply(lambda x: division(x["Strong Roof/Strong Wall"], x["Housing Units"]), axis=1).values
df_total["Strong Roof/Light Wall"] = df_total.apply(lambda x: division(x["Strong Roof/Strong Wall"], x["Housing Units"]), axis=1).values
df_total["Strong Roof/Salvage Wall"] = df_total.apply(lambda x: division(x["Strong Roof/Strong Wall"], x["Housing Units"]), axis=1).values
df_total["Light Roof/Strong Wall"] = df_total.apply(lambda x: division(x["Strong Roof/Strong Wall"], x["Housing Units"]), axis=1).values
df_total["Light Roof/Light Wall"] = df_total.apply(lambda x: division(x["Strong Roof/Strong Wall"], x["Housing Units"]), axis=1).values
df_total["Light Roof/Salvage Wall"] = df_total.apply(lambda x: division(x["Strong Roof/Strong Wall"], x["Housing Units"]), axis=1).values
df_total["Salvaged Roof/Strong Wall"] = df_total.apply(lambda x: division(x["Strong Roof/Strong Wall"], x["Housing Units"]), axis=1).values
df_total["Salvaged Roof/Light Wall"] = df_total.apply(lambda x: division(x["Strong Roof/Strong Wall"], x["Housing Units"]), axis=1).values
df_total["Salvaged Roof/Salvage Wall"] = df_total.apply(lambda x: division(x["Strong Roof/Strong Wall"], x["Housing Units"]), axis=1).values
 

,Mun_Code,typhoon,rainfall_Total,rainfall_max_6h,rainfall_max_24h,v_max,dis_track_min,id,Year,Totally,...,Salvaged Roof/Salvage Wall,vulnerable_groups,Female,Male,Total Pop,Total # of Active HHs,Total HH MEM_M,Total HH MEM_F,perc_dmg,pantawid_pamilya_beneficiary
1,PH175101000,durian2006,182.760714,14.716071,7.381696,55.032241,2.478142,30.0,2006.0,261.0,...,0.000064,0.029515,15074,16232,31306,3372.0,9337.0,8671.0,0.036326,0.469311
2,PH030801000,durian2006,27.012500,1.893750,1.070833,23.402905,136.527982,NaN,NaN,0.0,...,0.000097,0.008676,19779,20101,39880,830.0,2321.0,2164.0,0.000000,0.089672
4,PH083701000,durian2006,0.137500,3.563542,1.050260,8.728380,288.358553,NaN,NaN,0.0,...,0.000037,0.033389,27840,31731,59571,3455.0,10000.0,8988.0,0.000000,0.259892
6,PH045601000,durian2006,159.666667,13.283333,6.165278,52.891105,41.386956,NaN,NaN,NaN,...,0.000217,0.019532,6415,6436,12851,594.0,1823.0,1723.0,NaN,0.188213
8,PH015501000,durian2006,23.100000,2.408333,0.957639,10.945624,274.953818,NaN,NaN,0.0,...,0.000090,0.021318,13808,14244,28052,2122.0,5613.0,5369.0,0.000000,0.321857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97190,PH025014000,noul2015,4.737500,0.614583,0.193750,15.791907,210.313249,NaN,NaN,0.0,...,0.000166,0.011401,9088,9419,18507,415.0,1325.0,1227.0,0.000000,0.093490
97191,PH140127000,noul2015,1.587500,0.122917,0.060417,13.867145,218.189328,NaN,NaN,0.0,...,0.000626,0.028375,2631,2761,5392,232.0,743.0,637.0,0.000000,0.219282
97193,PH012823000,noul2015,34.327273,2.940152,0.855628,18.174500,154.588189,NaN,NaN,0.0,...,0.000119,0.006921,15850,16370,32220,907.0,2713.0,2504.0,0.000000,0.118007
97194,PH051612000,noul2015,0.644444,0.062963,0.018056,15.647639,219.542224,NaN,NaN,0.0,...,0.000049,0.025181,21293,22192,43485,3047.0,9547.0,8654.0,0.000000,0.316341


### filter final data frame for only relevant features 

In [54]:
df_total=df_total.filter(['Mun_Code', 'typhoon', 'rainfall_Total', 'rainfall_max_6h',
                 'rainfall_max_24h', 'v_max', 'dis_track_min','landslide_per', 'stormsurge_per',
                 'Bu_p_inSSA','Bu_p_LS', 'Red_per_LSbldg', 'Or_per_LSblg',
                 'Yel_per_LSSAb','RED_per_SSAbldg', 'OR_per_SSAbldg', 
                 'Yellow_per_LSbl', 'mean_slope','mean_elevation_m', 
                 'ruggedness_stdev', 'mean_ruggedness','slope_stdev',
                 'poverty_perc', 'with_coast', 'coast_length','Housing Units', 'Strong Roof/Strong Wall',
                 'Strong Roof/Light Wall', 'Strong Roof/Salvage Wall',
                 'Light Roof/Strong Wall', 'Light Roof/Light Wall',
                 'Light Roof/Salvage Wall', 'Salvaged Roof/Strong Wall',
                 'Salvaged Roof/Light Wall', 'Salvaged Roof/Salvage Wall',
                 'vulnerable_groups', 'pantawid_pamilya_beneficiary','perc_dmg'])


### save final dataframe for model input

In [55]:
df_total.to_csv("IBF-typhoon-model\\data\\combined_input_data.csv", index=False)